In [7]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x :.2f}')
pd.set_option('display.max_columns', None)
from cleaner import DataCleaner
from get_dataframes import GetDataframes
import hjson

from datetime import datetime

import matplotlib.pyplot as plt
import explo_data_analysis.eda_movies as eda

with open("config.hjson") as fp:
    config = hjson.load(fp)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
tmdb = "clean_datasets/tmdb_updated.parquet"
actors = "clean_datasets/actors_movies.parquet"
directors = "clean_datasets/directors_movies.parquet"
movies = "clean_datasets/movies_cleaned.parquet"

tmdb = pd.read_parquet(tmdb)
actors = pd.read_parquet(actors)
directors = pd.read_parquet(directors)
movies = pd.read_parquet(movies)

In [271]:
# actors[actors["titre_str"].str.contains("Spider-Man")]

In [12]:
movies

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,region,cuts
0,tt0035423,Kate & Leopold,2001,118,"[Comedy, Fantasy, Romance]",6.40,87897,en,Kate & Leopold,15.77,[US],76019048.00,"[en, fr, it]",Released,FR,2000-2009
1,tt0052832,The Fugitive Kind,1960,119,"[Drama, Romance]",7.10,6914,en,The Fugitive Kind,5.98,[US],0.00,[en],Released,FR,1960-1969
2,tt0052997,The League of Gentlemen,1960,116,"[Comedy, Crime, Drama]",7.20,5105,en,The League of Gentlemen,5.16,[GB],0.00,[en],Released,FR,1960-1969
3,tt0053459,Eyes Without a Face,1960,90,"[Drama, Horror]",7.60,34278,fr,Les Yeux sans visage,9.05,"[FR, IT]",58656.00,[fr],Released,FR,1960-1969
4,tt0053472,Breathless,1960,90,"[Crime, Drama]",7.70,86230,fr,À bout de souffle,13.45,[FR],0.00,"[fr, en]",Released,FR,1960-1969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8287,tt9873892,They Cloned Tyrone,2023,122,"[Comedy, Mystery, Sci-Fi]",6.60,33197,en,They Cloned Tyrone,25.08,[US],0.00,[en],Released,FR,>2020
8289,tt9883996,Dream Horse,2020,113,"[Biography, Comedy, Drama]",6.90,6589,en,Dream Horse,12.54,"[GB, US]",6435260.00,[en],Released,FR,>2020
8290,tt9893250,I Care a Lot,2020,118,"[Comedy, Crime, Drama]",6.40,141372,en,I Care a Lot,22.05,"[GB, US]",0.00,[en],Released,FR,>2020
8291,tt9900782,Kaithi,2019,145,"[Action, Adventure, Crime]",8.50,39546,ta,கைதி,9.74,[IN],0.00,"[ta, te]",Released,FR,2010-2019


In [9]:
col_to_keep = [
    "titre_id",
    "titre_str",
    "titre_genres",
    "rating_avg",
    "rating_votes"
]
movies = movies[col_to_keep]

In [10]:
directors_list_id = directors["titre_id"]

condi = movies["titre_id"].isin(directors_list_id)
condi2 = actors["titre_id"].isin(directors_list_id)

movies = movies[condi]
actors = actors[condi2]

In [5]:
actors_list_id = actors["titre_id"]

condi = movies["titre_id"].isin(actors_list_id)
condi2 = directors["titre_id"].isin(actors_list_id)

movies = movies[condi]
directors = directors[condi2]

In [6]:
print(len(directors_list_id))
print(len(actors_list_id))
print(len(movies))


8303
30755
7656


In [275]:
col_to_keep = [
    "imdb_id",
    "overview"
]

tmdb = tmdb[col_to_keep]

In [276]:
col_to_keep = [
    "titre_id",
    "person_name",
    # "person_index"
]

actors = actors[col_to_keep]

In [277]:
col_to_keep = [
    "titre_id",
    "person_name",
    # "person_index"
]
directors = directors[col_to_keep]

In [278]:
actors.loc[:, "person_name"] = actors["person_name"].str.split(", ")
actors

,titre_id,person_name
0,tt0093191,[Curt Bois]
1,tt0093191,[Bruno Ganz]
2,tt0093191,[Otto Sander]
3,tt0093191,[Solveig Dommartin]
4,tt0057643,[José Isbert]
...,...,...
31102,tt8267604,[Kawsar Al Haddad]
31103,tt4028464,[Rakel Lenora Petersen Fløttum]
31104,tt4028464,[Rakel Lenora Petersen Fløttum]
31105,tt4028464,[Sam Ashraf]


In [279]:
# split_genres = nodup["knowForTitles"].str.split(",", expand=True)
# split_genres.columns = [f'knowForTitles_{i+1}' for i in range(split_genres.shape[1])]
# df1 = pd.concat([nodup.drop(columns='knowForTitles'), split_genres], axis=1)
# 25 octobre 2023
# actors

In [280]:
# actors = actors.sort_values(by=["titre_id", "person_index"])
# actors = actors.pivot(index="titre_id", columns="person_index", values="person_name").reset_index()
# actors.columns = [f"actor_{i}" for i in actors.columns]

In [281]:
# actors.drop(columns=["actor_6", "actor_7", "actor_8", "actor_9", "actor_10"], axis=1, inplace=True)
# actors.rename(columns={"actor_titre_id": "titre_id"}, inplace=True)
# actors

In [282]:
# def str_to_lists(
#     df: pd.DataFrame,
#     col_to_keep_: list
# ):
#     df = df[col_to_keep_]
#     df.loc[:, "person_name"] = df["person_name"].str.split(", ")
#     name = df.groupby("titre_id")["person_name"].sum().reset_index()
#     return name["person_name"].to_list()

In [283]:
directors.loc[:, "person_name"] = directors["person_name"].str.split(", ")

In [284]:
actors

,titre_id,person_name
0,tt0093191,[Curt Bois]
1,tt0093191,[Bruno Ganz]
2,tt0093191,[Otto Sander]
3,tt0093191,[Solveig Dommartin]
4,tt0057643,[José Isbert]
...,...,...
31102,tt8267604,[Kawsar Al Haddad]
31103,tt4028464,[Rakel Lenora Petersen Fløttum]
31104,tt4028464,[Rakel Lenora Petersen Fløttum]
31105,tt4028464,[Sam Ashraf]


In [285]:
# # actors
person_name = actors.groupby("titre_id")["person_name"].sum().reset_index()
person_list = person_name["person_name"].to_list()

In [286]:
# directors
directors_name = directors.groupby("titre_id")["person_name"].sum().reset_index()
directors_list = directors_name["person_name"].to_list()

In [287]:
actors

,titre_id,person_name
0,tt0093191,[Curt Bois]
1,tt0093191,[Bruno Ganz]
2,tt0093191,[Otto Sander]
3,tt0093191,[Solveig Dommartin]
4,tt0057643,[José Isbert]
...,...,...
31102,tt8267604,[Kawsar Al Haddad]
31103,tt4028464,[Rakel Lenora Petersen Fløttum]
31104,tt4028464,[Rakel Lenora Petersen Fløttum]
31105,tt4028464,[Sam Ashraf]


In [288]:
movies["actors"] = person_list
movies["directors"] = directors_list
movies

,titre_id,titre_str,titre_genres,rating_avg,rating_votes,actors,directors
0,tt0035423,Kate & Leopold,"[Comedy, Fantasy, Romance]",6.40,87897,"[Meg Ryan, Hugh Jackman, Liev Schreiber, Breck...",[James Mangold]
1,tt0052832,The Fugitive Kind,"[Drama, Romance]",7.10,6914,"[Anna Magnani, Marlon Brando, Joanne Woodward,...",[Sidney Lumet]
2,tt0052997,The League of Gentlemen,"[Comedy, Crime, Drama]",7.20,5105,"[Jack Hawkins, Roger Livesey, Nigel Patrick, R...",[Basil Dearden]
3,tt0053459,Eyes Without a Face,"[Drama, Horror]",7.60,34278,"[Pierre Brasseur, Alexandre Rignault, Alida Va...",[Georges Franju]
5,tt0053579,Late Autumn,"[Comedy, Drama]",7.90,5854,"[Setsuko Hara, Keiji Sada, Mariko Okada, Yôko ...",[Yasujirô Ozu]
...,...,...,...,...,...,...,...
8288,tt9873892,They Cloned Tyrone,"[Comedy, Mystery, Sci-Fi]",6.60,33197,"[Kiefer Sutherland, Jamie Foxx, John Boyega, T...",[Juel Taylor]
8290,tt9883996,Dream Horse,"[Biography, Comedy, Drama]",6.90,6589,"[Alan David, Lynda Baron, Owen Teale, Toni Col...",[Euros Lyn]
8291,tt9893250,I Care a Lot,"[Comedy, Crime, Drama]",6.40,141372,"[Dianne Wiest, Rosamund Pike, Peter Dinklage, ...",[J Blakeson]
8292,tt9900782,Kaithi,"[Action, Adventure, Crime]",8.50,39546,"[Narain, Karthi, George Maryan, Arjun Das]",[Lokesh Kanagaraj]


In [289]:
merged = pd.merge(
    movies,
    tmdb,
    left_on = "titre_id",
    right_on = "imdb_id"
)

# merged_ = pd.merge(
#     merged,
#     actors,
#     on="titre_id"
# )
# merged_

In [290]:
merged

,titre_id,titre_str,titre_genres,rating_avg,rating_votes,actors,directors,imdb_id,overview
0,tt0035423,Kate & Leopold,"[Comedy, Fantasy, Romance]",6.40,87897,"[Meg Ryan, Hugh Jackman, Liev Schreiber, Breck...",[James Mangold],tt0035423,When her scientist ex-boyfriend discovers a po...
1,tt0052832,The Fugitive Kind,"[Drama, Romance]",7.10,6914,"[Anna Magnani, Marlon Brando, Joanne Woodward,...",[Sidney Lumet],tt0052832,"Val Xavier, a drifter of obscure origins, arri..."
2,tt0052997,The League of Gentlemen,"[Comedy, Crime, Drama]",7.20,5105,"[Jack Hawkins, Roger Livesey, Nigel Patrick, R...",[Basil Dearden],tt0052997,Involuntarily-retired Colonel Hyde recruits se...
3,tt0053459,Eyes Without a Face,"[Drama, Horror]",7.60,34278,"[Pierre Brasseur, Alexandre Rignault, Alida Va...",[Georges Franju],tt0053459,Dr. Génessier is riddled with guilt after an a...
4,tt0053579,Late Autumn,"[Comedy, Drama]",7.90,5854,"[Setsuko Hara, Keiji Sada, Mariko Okada, Yôko ...",[Yasujirô Ozu],tt0053579,A woman and her daughter are each forced to co...
...,...,...,...,...,...,...,...,...,...
7411,tt9873892,They Cloned Tyrone,"[Comedy, Mystery, Sci-Fi]",6.60,33197,"[Kiefer Sutherland, Jamie Foxx, John Boyega, T...",[Juel Taylor],tt9873892,A series of eerie events thrusts an unlikely t...
7412,tt9883996,Dream Horse,"[Biography, Comedy, Drama]",6.90,6589,"[Alan David, Lynda Baron, Owen Teale, Toni Col...",[Euros Lyn],tt9883996,"The inspiring true story of Dream Alliance, an..."
7413,tt9893250,I Care a Lot,"[Comedy, Crime, Drama]",6.40,141372,"[Dianne Wiest, Rosamund Pike, Peter Dinklage, ...",[J Blakeson],tt9893250,A court-appointed legal guardian defrauds her ...
7414,tt9900782,Kaithi,"[Action, Adventure, Crime]",8.50,39546,"[Narain, Karthi, George Maryan, Arjun Das]",[Lokesh Kanagaraj],tt9900782,"Dilli, a convicted criminal, is out on parole ..."


In [291]:
# merged.drop(["titre_id", "imdb_id"], axis = 1, inplace = True)
merged.drop(["imdb_id"], axis = 1, inplace = True)

In [ ]:
merged.to_parquet("machine_learning_v2.parquet")